# Run this in Kaggle

# 🚗 Vehicle Damage Detection + Cost Estimation  
### Part 1 — YOLOv5 Training (Kaggle)

This notebook prepares and trains a **YOLOv5 damage detector** using the *Car Damage Dataset (CarDD)*.  
In **Part 2**, we will integrate this model in Colab with an **AI-based repair cost estimator**.

---

# 📦 Dataset Used
Kaggle Dataset:  
https://www.kaggle.com/datasets/huebitsvizg/vehicle-damage

Inside this dataset you receive:
CarDD_release/CarDD_COCO/
├── train2017/
├── val2017/
├── test2017/
└── annotations/ instances_*.json (COCO format)


The annotations are in **COCO JSON format**.  
YOLOv5 requires:

images/train
images/val
images/test
labels/train
labels/val
labels/test
cardd.yaml


So the first part of the pipeline converts **COCO → YOLO labels**.

---

# 🧠 What This Notebook Will Do

### ✔ Convert COCO annotations → YOLO labels  
### ✔ Generate required folder structure  
### ✔ Train YOLOv5s for damage detection  
### ✔ Run inference on test images  
### ✔ Export trained model for Colab App (Part 2)

After training, the following file becomes important:

runs/train/CarDD/weights/best.pt


You will upload this to Google Drive during the Colab web app phase.

---

# 🛠 NOTES
• Only **1 class** is used: `"damage"`  
• Later in Colab, we’ll layer a **cost estimation model** on top of the detector output  
• This Kaggle notebook focuses on **object detection only**  


In [ ]:
!pip install --force-reinstall "numpy==1.26.4" "opencv-python==4.7.0.72" "matplotlib==3.7.2"
!rm -rf yolov5
!git clone https://github.com/ultralytics/yolov5.git
%cd yolov5
!pip install -r requirements.txt --quiet

# 📁 Step 1 — Prepare YOLO Directory Structure

YOLOv5 expects the dataset in this layout:

CarDD_yolo/
├── images/train  
├── images/val  
├── images/test  
├── labels/train  
├── labels/val  
├── labels/test  
└── cardd.yaml

This block creates these folders inside `/kaggle/working`.

The *original dataset* stays in:
`/kaggle/input/damage-dataset/CarDD_release/CarDD_COCO/`


In [ ]:
import json, shutil, os
from pathlib import Path

# Paths
root = Path("/kaggle/input/damage-dataset/CarDD_release/CarDD_COCO")
out_root = Path("/kaggle/working/CarDD_yolo")

# Create folders
for split in ["train", "val", "test"]:
    (out_root / "images" / split).mkdir(parents=True, exist_ok=True)
    (out_root / "labels" / split).mkdir(parents=True, exist_ok=True)


# 🔄 Step 2 — Convert COCO JSON → YOLO Labels

The COCO annotation format gives:
- top-left (x, y)
- width
- height

YOLO requires:
- class_id  
- center_x  
- center_y  
- width  
- height  
(all normalized)

Also:
✔ Reads image sizes using OpenCV  
✔ Writes YOLO `.txt` label files  
✔ Copies images into YOLO folders  


In [ ]:
def coco_to_yolo(json_path, image_dir, out_img, out_lbl):
    with open(json_path) as f:
        data = json.load(f)

    # Map image-id → filename
    id_to_file = {img["id"]: img["file_name"] for img in data["images"]}

    # Collect annotations per image
    anns = {}
    for a in data["annotations"]:
        img_id = a["image_id"]
        x, y, w, h = a["bbox"]

        # Convert bbox to YOLO format (normalized center x,y,w,h)
        cx = (x + w/2)
        cy = (y + h/2)
        img_file = image_dir / id_to_file[img_id]
        img_path = str(img_file)

        import cv2
        img = cv2.imread(img_path)
        H, W = img.shape[:2]

        cx /= W
        cy /= H
        w /= W
        h /= H

        if img_id not in anns:
            anns[img_id] = []
        anns[img_id].append([0, cx, cy, w, h])  # class: 0 (damage)

    # Write image files & labels
    for img_id, file_name in id_to_file.items():
        src_img = image_dir / file_name
        dst_img = out_img / file_name
        shutil.copy(src_img, dst_img)

        label_file = out_lbl / (file_name.replace(".jpg", ".txt"))
        with open(label_file, "w") as f:
            for anno in anns.get(img_id, []):
                f.write(" ".join(map(str, anno)) + "\n")


# 🧩 Step 3 — Run COCO → YOLO Conversion for Train/Val/Test

Each annotation file is converted automatically.

After this step:
✔ All images copied  
✔ YOLO label files created  
✔ Dataset ready for YOLOv5


In [ ]:
coco_to_yolo(
    json_path=root/"annotations/instances_train2017.json",
    image_dir=root/"train2017",
    out_img=out_root/"images/train",
    out_lbl=out_root/"labels/train"
)

coco_to_yolo(
    json_path=root/"annotations/instances_val2017.json",
    image_dir=root/"val2017",
    out_img=out_root/"images/val",
    out_lbl=out_root/"labels/val"
)

coco_to_yolo(
    json_path=root/"annotations/instances_test2017.json",
    image_dir=root/"test2017",
    out_img=out_root/"images/test",
    out_lbl=out_root/"labels/test"
)


# 📝 Step 4 — Create cardd.yaml (YOLO Config File)

YOLOv5 requires a dataset configuration YAML file containing:

train: path/to/train  
val: path/to/val  
test: path/to/test  
nc: number of classes  
names: ["damage"]

This file tells YOLO how many classes and where the dataset lives.


In [ ]:
import os, shutil, json
from tqdm import tqdm
import yaml

base = "/kaggle/working/CarDD_yolo"
os.makedirs(base, exist_ok=True)

# Write cardd.yaml
yaml.safe_dump({
    "train": f"{base}/images/train",
    "val": f"{base}/images/val",
    "test": f"{base}/images/test",
    "nc": 1,
    "names": ["damage"]
}, open(f"{base}/cardd.yaml", "w"))

print("✔ cardd.yaml created at", f"{base}/cardd.yaml")


# ⚠️ Step 5 — Kaggle Fixes for TensorFlow / Protobuf Conflicts

Kaggle occasionally auto-installs packages that break YOLOv5.

We disable:
✔ TensorBoard  
✔ protobuf  
✔ W&B  

This ensures YOLOv5 trains without errors.


In [ ]:
# FIX 1: Disable TensorBoard entirely (avoids protobuf errors)
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['WANDB_DISABLED'] = 'true'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# FIX 2: Uninstall TensorBoard + protobuf if already installed
!pip uninstall -y tensorboard protobuf


# 🚀 Step 6 — Train YOLOv5s Model

Training settings:
• Image size: 640  
• Batch size: 16  
• Epochs: 50  
• Base model: yolov5s.pt  
• Project name: CarDD  

After training, your important model file is:

📌 runs/train/CarDD/weights/best.pt  
(This will be used in the Colab Damage Cost Estimator App)


In [ ]:
!python train.py \
  --img 640 \
  --batch 16 \
  --epochs 50 \
  --data /kaggle/working/CarDD_yolo/cardd.yaml \
  --weights yolov5s.pt \
  --name CarDD \
  --exist-ok


# 🔍 Step 7 — Run Test Inference

This runs YOLOv5 detection on all test images.

Outputs stored in:

runs/detect/CarDD_test/

Useful for:
✔ Validating detection quality  
✔ Exporting predictions  
✔ Preparing data for damage-cost estimation model


In [ ]:
!python detect.py \
  --weights runs/train/CarDD/weights/best.pt \
  --source /kaggle/working/CarDD_yolo/images/test \
  --img 640 \
  --conf 0.25 \
  --name CarDD_test


# 🖼 Step 8 — Display Sample Predictions

Shows annotated images with bounding boxes.

This confirms:
• COCO → YOLO conversion worked  
• Model trained correctly  
• Predictions are visually reasonable  

Next Step:
You will upload **best.pt** to Colab for the **Damage + Cost Estimation Web App**.


In [ ]:
import glob
from IPython.display import Image

for img in glob.glob('runs/detect/CarDD_test/*.jpg')[:5]:
    display(Image(filename=img))


# Run this in Colab




---

# 🚗 **Vehicle Damage Detection + Cost Estimation (Web App – Colab Deployment)**

This notebook allows you to deploy a **full web application** for detecting vehicle damage and estimating repair cost using:

* **YOLOv5** → Damage detection
* **LLaVA 1.6 Mistral 7B** → Car-part reasoning + damage description
* **Flask Web App** → User interface for uploading images & estimating cost

This Colab notebook is the **second stage** of the project.

---

## 🔗 **Prerequisites from Kaggle Training Notebook**

Before running this Colab web app, you MUST complete the **Kaggle training notebook**, where you:

✔ Train YOLOv5 on the Vehicle Damage dataset
✔ Export the trained weights
✔ Obtain the file:

```
best.pt   (your custom YOLO damage detector)
```

### ⭐ REQUIRED ACTION

Download the `best.pt` model from Kaggle and upload it to:

```
Google Drive → MyDrive → vehicle damage detection/
```

The web app will load your trained model from:

```
/content/drive/MyDrive/vehicle damage detection/best (2).pt
```

(You can rename your file or update the path in `YOLO_MODEL_PATH` accordingly.)

---

## 🔐 **Hugging Face Token Requirement**

This app uses **LLaVA-1.6 Mistral 7B**, which requires authentication to download.

### How to create a token:

1. Visit [https://huggingface.co/settings/tokens](https://huggingface.co/settings/tokens)
2. Click **New Token → Read Access**
3. Copy the token (starts with `hf_...`)

Paste it inside:

```python
login(token="hf_your_token_here")
```

---

## 📁 **Project Workflow Overview**

```
USER IMAGE → YOLO → detected bboxes → crop regions
                         ↓
                 LLaVA (Vision + Language)
                         ↓
        Car part names + short damage description
                         ↓
      Cost estimation engine (rule + severity-based)
                         ↓
              Final formatted repair report
```

The web app consists of two stages:

### 1️⃣ /api/analyze_damage

* Applies YOLO model
* Returns:

  * annotated bounding-box image
  * list of detections
  * severity score (avg conf)
  * tightly cropped damage images (used by LLaVA)

### 2️⃣ /api/estimate_cost

* Runs LLaVA on each crop
* Identifies:

  * affected part (e.g., "front bumper")
  * nature of damage (e.g., "dented")
* Estimates repair cost
* Produces a formatted plain-text repair report

---

## 📦 **Colab Setup Explained**

### **1. Mount Google Drive**

Used to load your trained YOLO model.

```python
drive.mount("/content/drive/")
```

### **2. Install dependencies**

YOLO, Flask, LLaVA transformers, BitsAndBytes, Pillow, ngrok.

```python
!pip install -q flask pyngrok ultralytics transformers ...
```

### **3. Create folders**

Required for Flask templates, static files, and user uploads.

```python
!mkdir -p templates static uploads
```

### **4. Authenticate Hugging Face**

Needed to load LLaVA.

```python
login(token="hf_...")
```

---

## 🧠 **Model Loading Explained**

### YOLO Model (Damage Detection)

Loaded with:

```python
torch.hub.load("ultralytics/yolov5", "custom", path=YOLO_MODEL_PATH)
```

### LLaVA Model (Car-Part Recognition)

Loaded with 8-bit quantization:

```python
LlavaForConditionalGeneration.from_pretrained(..., quantization_config=BNB_CONFIG)
```

Benefits:

* Lower VRAM usage
* Faster inference

---

## 🔍 **Damage Detection Logic (YOLO Section)**

When the user uploads a vehicle image:

1. YOLO detects damage regions
2. Very small boxes are filtered out
3. Each bounding box is cropped tightly
4. Severity score = average YOLO confidence
5. Annotated image is generated for preview

---

## 🧠 **Car Part Identification Logic (LLaVA Section)**

Each crop is passed to LLaVA with a structured prompt:

```
Identify the EXACT car part and describe the damage.
Return only: <part> - <damage>
```

Examples:

```
Front bumper – cracked
Right headlight – broken
Left fender – major dent
```

LLaVA output is cleaned, deduplicated, and limited to max 8 parts.

---

## 💰 **Cost Estimation Engine Logic**

Cost is calculated using:

### Inputs:

* Number of damages
* Severity score
* Type of parts damaged

### Output:

A cost range, e.g.:

```
₹24,500 – ₹58,000
```

---

## 🧾 **Final Report Format**

The backend returns a professional repair report:

```
Damage Level: Moderate

Estimated Repair Cost (INR):
₹18,000 – ₹42,000

Parts Affected:
- Front bumper - dented
- Left headlight - cracked

Repair Recommendation:
- Replace damaged components
- Repaint affected areas
- Perform alignment check
```

---

## 🖥 **Web UI Explanation**

Your app includes:

### **/ (Home page)**

Introduction + navigation

### **/estimate page**

Upload → Detection → Severity → Crops → LLaVA → Cost Estimate

### All interactions happen without page reload

(using fetch APIs).

---

## ▶️ **Starting the Web App**

At the end of the notebook:

```python
!nohup python app.py > flask.log 2>&1 &
```

The server runs in the background.

### Then you start ngrok:

```python
public_url = ngrok.connect(8000)
```

Copy the public URL → open in browser → start using the app!




In [ ]:
from google.colab import drive
drive.mount("/content/drive/")

In [ ]:
!ls "/content/drive/MyDrive/vehicle damage detection"

In [ ]:
!pip install -q flask pyngrok ultralytics transformers accelerate bitsandbytes pillow


In [ ]:
!mkdir -p templates static uploads


In [ ]:
from huggingface_hub import login
login(token="hf_your_token_here")   # replace


In [ ]:
%%writefile templates/navbar.html
<header class="topbar">
  <div class="topbar-left">
    <span class="logo-icon">🚗</span>
    <div class="brand-box">
      <div class="brand-title">AutoDamage AI</div>
      <div class="brand-sub">Smart Vehicle Damage Estimation</div>
    </div>
  </div>

  <div class="topbar-right">
    <a href="{{ url_for('home') }}"
       class="nav-btn {{ 'active' if request.path in ['/', url_for('home')] else '' }}">
       Home
    </a>

    <a href="{{ url_for('estimate_page') }}"
       class="nav-btn {{ 'active' if request.path == url_for('estimate_page') else '' }}">
       Estimate Cost
    </a>
  </div>
</header>


In [ ]:
%%writefile app.py
import os
import uuid
import json
import math
from io import BytesIO
from PIL import Image
from pathlib import Path

from flask import Flask, render_template, request, send_from_directory, jsonify
from werkzeug.utils import secure_filename

import torch
from transformers import BitsAndBytesConfig

# LLaVA imports
from transformers import LlavaProcessor, LlavaForConditionalGeneration

# YOLOv5 via torch.hub
import torchvision.transforms as T

# -------------------------------------------
# FLASK CONFIG
# -------------------------------------------
app = Flask(__name__, template_folder="templates", static_folder="static")
UPLOAD_DIR = Path("uploads")
UPLOAD_DIR.mkdir(parents=True, exist_ok=True)

# -------------------------------------------
# MODEL PATHS
# -------------------------------------------
YOLO_MODEL_PATH = "/content/drive/MyDrive/vehicle damage detection/best (2).pt"
LLAVA_MODEL = "llava-hf/llava-v1.6-mistral-7b-hf"

# Cache
CACHE = {"detector": None, "processor": None, "llava": None}

# GPU config
BNB_CONFIG = BitsAndBytesConfig(load_in_8bit=True)

# -------------------------------------------
# LOAD MODELS
# -------------------------------------------
def load_models(force_reload=False):
    # Load YOLOv5 detector via torch.hub (custom)
    if CACHE["detector"] is None or force_reload:
        try:
            CACHE["detector"] = torch.hub.load(
                "ultralytics/yolov5",
                "custom",
                path=YOLO_MODEL_PATH,
                force_reload=False
            )
        except Exception as e:
            print("YOLO load failed:", e)
            CACHE["detector"] = None

    # Load LLaVA processor + model
    if CACHE["processor"] is None or CACHE["llava"] is None or force_reload:
        try:
            CACHE["processor"] = LlavaProcessor.from_pretrained(LLAVA_MODEL)
            CACHE["llava"] = LlavaForConditionalGeneration.from_pretrained(
                LLAVA_MODEL,
                device_map="cuda",
                torch_dtype=torch.float16,
                quantization_config=BNB_CONFIG,
                low_cpu_mem_usage=True
            )
        except Exception as e:
            print("LLaVA load failed:", e)
            CACHE["processor"] = None
            CACHE["llava"] = None

    return CACHE["detector"], CACHE["processor"], CACHE["llava"]

# -------------------------------------------
# TIGHT CROP FUNCTION
# -------------------------------------------
def crop_image_tight(pil_img, bbox):
    x1, y1, x2, y2 = map(int, bbox)
    x1 = max(0, x1)
    y1 = max(0, y1)
    x2 = min(pil_img.width - 1, x2)
    y2 = min(pil_img.height - 1, y2)
    if x2 <= x1 or y2 <= y1:
        return None
    return pil_img.crop((x1, y1, x2, y2))

# -------------------------------------------
# IDENTIFY PART USING LLaVA (IMPROVED)
# -------------------------------------------
def identify_part_with_llava(processor, model, crop_pil):
    """
    Uses LLaVA to identify specific car part and short damage in format:
    <Part name> - <damage>
    Returns None on failure.
    """
    try:
        # ensure RGB and resize for stable performance
        crop_pil = crop_pil.convert("RGB").resize((336, 336))

        prompt = (
            "You are an automotive damage expert.\n"
            "Identify the EXACT car part visible in this image and describe the damage.\n"
            "Return ONLY one short line in this EXACT format:\n"
            "<part name> - <damage>\n\n"
            "Examples:\n"
            "Front bumper - cracked\n"
            "Right headlight - broken\n"
            "Wheel - punctured\n"
            "Left fender - dented\n"
        )

        # Processor expects (images, text)
        inputs = processor(crop_pil, prompt, return_tensors="pt").to(model.device, torch.float16)

        with torch.inference_mode():
            out = model.generate(
                **inputs,
                max_new_tokens=60,
                do_sample=False,
                eos_token_id=processor.tokenizer.eos_token_id if hasattr(processor, "tokenizer") else None
            )

        text = processor.decode(out[0], skip_special_tokens=True).strip()
        if not text:
            return None

        # Take first non-empty line
        line = next((ln.strip() for ln in text.splitlines() if ln.strip()), "").strip()

        # cleanup
        line = line.replace("Answer:", "").replace("###", "").strip()
        # ensure contains dash to match format
        if "-" not in line:
            return None

        # limit length
        return line[:140]
    except Exception as e:
        # print for debug in flask logs
        print("LLaVA error:", e)
        return None

# -------------------------------------------
# COST ENGINE
# -------------------------------------------
def compute_cost_range(num_damages, severity):
    if num_damages == 0:
        base_low, base_high = 2000, 6000
    elif num_damages == 1:
        base_low, base_high = 8000, 15000
    elif num_damages == 2:
        base_low, base_high = 15000, 30000
    elif num_damages == 3:
        base_low, base_high = 25000, 60000
    else:
        base_low, base_high = 40000, 120000

    multiplier = 1.0 + (severity * 1.2)
    low = int(math.ceil(base_low * multiplier))
    high = int(math.ceil(base_high * multiplier))
    return f"₹{low:,} – ₹{high:,}"

# -------------------------------------------
# FORMAT OUTPUT (F1)
# -------------------------------------------
def format_report(damage_level, cost_range, parts_list):
    parts_text = "\n".join(f"- {p}" for p in parts_list) if parts_list else "- No specific part identified"
    recommendations = [
        "Replace damaged components",
        "Repaint affected areas",
        "Perform alignment check"
    ]
    rec_text = "\n".join(f"- {r}" for r in recommendations)

    report = (
        f"Damage Level: {damage_level}\n\n"
        f"Estimated Repair Cost (INR):\n{cost_range}\n\n"
        f"Parts Affected:\n{parts_text}\n\n"
        f"Repair Recommendation:\n{rec_text}"
    )
    return report

# -------------------------------------------
# ROUTES
# -------------------------------------------
@app.route("/")
def home():
    return render_template("index.html")

@app.route("/estimate")
def estimate_page():
    return render_template("estimate.html")

@app.route("/uploads/<path:filename>")
def uploaded_file(filename):
    return send_from_directory(str(UPLOAD_DIR), filename)

# -------------------------------------------
# API: ANALYZE DAMAGE (YOLO)
# -------------------------------------------
@app.route("/api/analyze_damage", methods=["POST"])
def analyze_damage_api():
    try:
        if "image" not in request.files:
            return jsonify({"error": "No image uploaded"}), 400

        img_file = request.files["image"]
        fname = f"{uuid.uuid4().hex}_{secure_filename(img_file.filename)}"
        saved_path = UPLOAD_DIR / fname
        img_file.save(saved_path)

        detector, _, _ = load_models()
        if detector is None:
            return jsonify({"error": "YOLO detector not loaded"}), 500

        results = detector(str(saved_path))

        try:
            boxes = results.xyxy[0].cpu().numpy()
        except:
            boxes = []

        detections = []
        confs = []
        crop_files = []

        pil_img = Image.open(saved_path).convert("RGB")

        for i, box in enumerate(boxes):
            x1, y1, x2, y2, conf, cls = box
            conf = float(conf)

            # ignore tiny boxes - likely noise
            if (x2 - x1) < 40 or (y2 - y1) < 40:
                continue

            detections.append({"class": "damage", "confidence": round(conf, 3)})
            confs.append(conf)

            crop = crop_image_tight(pil_img, (x1, y1, x2, y2))
            if crop is None:
                continue

            crop_name = f"{saved_path.stem}_crop_{i}.jpg"
            crop_path = UPLOAD_DIR / crop_name
            # save high-quality crop
            crop.save(crop_path, quality=90)
            crop_files.append(str(crop_path))

        severity = round(sum(confs) / len(confs), 3) if confs else 0.0

        try:
            results.render()
            rendered = results.ims[0]
            annotated_img = Image.fromarray(rendered)
            annotated_name = f"det_{fname}"
            annotated_path = UPLOAD_DIR / annotated_name
            annotated_img.save(annotated_path, quality=90)
            annotated_url = f"/uploads/{annotated_name}"
        except:
            annotated_url = None

        return jsonify({
            "success": True,
            "original": f"/uploads/{fname}",
            "annotated": annotated_url,
            "detections": detections,
            "severity": severity,
            "crops": crop_files
        })

    except Exception as e:
        return jsonify({"error": str(e)}), 500

# -------------------------------------------
# API: ESTIMATE COST (LLaVA)
# -------------------------------------------
@app.route("/api/estimate_cost", methods=["POST"])
def estimate_cost_api():
    try:
        payload = request.get_json(force=True)
        detections = payload.get("detections", [])
        severity = float(payload.get("severity", 0))
        crop_files = payload.get("crops", [])

        # load models
        detector, processor, llava = load_models()
        num_damages = len(detections)

        if severity < 0.30:
            damage_level = "Minor"
        elif severity < 0.65:
            damage_level = "Moderate"
        else:
            damage_level = "Severe"

        cost_range = compute_cost_range(num_damages, severity)

        parts_identified = []

        # fallback default mapping if llava not available or no crops
        if llava is None or processor is None or not crop_files:
            default_parts = {
                0: [],
                1: ["Front bumper - dented"],
                2: ["Front bumper - dented", "Left fender - scratched"],
                3: ["Front bumper - dented", "Left fender - scratched", "Headlight - broken"],
            }
            parts_identified = default_parts.get(num_damages, ["Front bumper - multiple panels"])
        else:
            # run LLaVA over each crop (tight crop)
            for cpath in crop_files:
                try:
                    crop_img = Image.open(cpath).convert("RGB")
                except Exception:
                    continue

                res = identify_part_with_llava(processor, llava, crop_img)
                if res:
                    cleaned = res.strip().rstrip(" .:;,-")
                    parts_identified.append(cleaned)

            # de-duplicate while preserving order using case-insensitive comparison
            seen = set()
            unique_parts = []
            for p in parts_identified:
                key = p.lower()
                if key not in seen:
                    unique_parts.append(p)
                    seen.add(key)
            parts_identified = unique_parts or ["Front bumper - multiple panels"]

        # limit number of parts
        parts_identified = parts_identified[:8]

        report = format_report(damage_level, cost_range, parts_identified)

        # return plain text report
        return app.response_class(report, mimetype="text/plain")

    except Exception as e:
        return jsonify({"error": str(e)}), 500

# -------------------------------------------
# RUN SERVER
# -------------------------------------------
if __name__ == "__main__":
    print("Loading models...")
    load_models()
    print("Models loaded. Starting server.")
    app.run(host="0.0.0.0", port=8000, debug=False)


In [ ]:
%%writefile templates/index.html
<!doctype html>
<html>
<head>
  <meta charset="utf-8">
  <title>AutoDamage AI</title>
  <link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}">
</head>

<body>
  {% include 'navbar.html' %}

  <main class="container">
    <section class="hero">
      <h1 class="title">AutoDamage AI</h1>
      <p class="subtitle">
        Advanced AI system using YOLO + LLaVA to analyze damage, identify affected parts,
        and generate a professional repair cost report.
      </p>

      <a href="{{ url_for('estimate_page') }}" class="cta-button">
        🚗 Start Damage Assessment
      </a>
    </section>
  </main>
</body>
</html>


In [ ]:
%%writefile templates/estimate.html
<!doctype html>
<html>
<head>
  <meta charset="utf-8">
  <title>Estimate Cost | AutoDamage AI</title>
  <link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}">
</head>

<body>

{% include 'navbar.html' %}

<main class="container">

  <h2 class="page-title">Vehicle Damage Assessment</h2>

  <div class="upload-box">
    <input type="file" id="image" accept="image/*" onchange="previewImage(event)">
    <img id="preview" class="preview-img" style="display:none">
  </div>

  <button id="analyzeBtn" class="btn primary" onclick="analyzeDamage()">🔍 Analyze Damage</button>
  <div id="loadingAnalyze" class="loader" style="display:none;">Analyzing damage…</div>

  <div id="resultSection" style="display:none">

    <h3>Detected Damages</h3>
    <div id="detList" class="det-list"></div>

    <h3>Severity Score</h3>
    <div id="sevScore"></div>

    <h3>Annotated Image</h3>
    <img id="annotatedImg" class="annotated-img">

    <h3>Cropped Damage Regions (Used for LLaVA part identification)</h3>
    <div id="cropGrid" class="crop-grid"></div>

    <button id="costBtn" class="btn accent" onclick="estimateCost()">💰 Generate Cost Estimate</button>
    <div id="loadingCost" class="loader" style="display:none;">Generating cost estimate…</div>

    <div id="costOutput" class="cost-box"></div>
  </div>

</main>

<script>
let uploadedFile = null;
let detections = [];
let severity = 0;
let cropFiles = [];   // IMPORTANT for LLaVA

function previewImage(e) {
  uploadedFile = e.target.files[0];
  const img = document.getElementById("preview");
  img.src = URL.createObjectURL(uploadedFile);
  img.style.display = "block";
}

async function analyzeDamage() {
  if (!uploadedFile) {
    alert("Select an image first!");
    return;
  }

  document.getElementById("analyzeBtn").disabled = true;
  document.getElementById("loadingAnalyze").style.display = "block";

  const fd = new FormData();
  fd.append("image", uploadedFile);

  const r = await fetch("/api/analyze_damage", { method: "POST", body: fd });
  const j = await r.json();

  document.getElementById("analyzeBtn").disabled = false;
  document.getElementById("loadingAnalyze").style.display = "none";

  detections = j.detections || [];
  severity = j.severity || 0;
  cropFiles = j.crops || [];   // <<--- IMPORTANT FIX

  document.getElementById("detList").innerHTML =
    detections.map(d => `<div>Class: ${d.class}, Conf: ${d.confidence}</div>`).join("");

  document.getElementById("sevScore").textContent = severity;
  document.getElementById("annotatedImg").src = j.annotated;

  // show cropped images
  document.getElementById("cropGrid").innerHTML =
    cropFiles.map(c => `<img src="${c}" class="crop-img">`).join("");

  document.getElementById("resultSection").style.display = "block";
}

async function estimateCost() {
  const payload = {
    detections,
    severity,
    crops: cropFiles   // <<--- SEND TO BACKEND
  };

  document.getElementById("costBtn").disabled = true;
  document.getElementById("loadingCost").style.display = "block";

  const r = await fetch("/api/estimate_cost", {
    method: "POST",
    headers: {"Content-Type": "application/json"},
    body: JSON.stringify(payload)
  });

  const text = await r.text();

  document.getElementById("costBtn").disabled = false;
  document.getElementById("loadingCost").style.display = "none";

  document.getElementById("costOutput").innerText = text;
}
</script>

</body>
</html>


In [ ]:
%%writefile static/style.css
/* =============================
   GLOBAL
==============================*/
body {
  background: #0b132b;
  color: white;
  font-family: Arial, sans-serif;
  margin: 0;
  padding: 0;
}

/* =============================
   NAVBAR
==============================*/
.topbar {
  display:flex;
  justify-content:space-between;
  padding:14px 24px;
  background:#1c2541;
  align-items:center;
  border-bottom: 2px solid #3a86ff22;
}

.nav-btn {
  color:#bfc7e0;
  margin-left:18px;
  text-decoration:none;
  transition:0.2s;
  padding: 6px 10px;
  border-radius: 6px;
}

.nav-btn:hover {
  color:white;
  background:#3a86ff33;
  transform:scale(1.08);
}

.nav-btn.active {
  color:white;
  background:#3a86ff55;
  font-weight:bold;
}

/* =============================
   PAGE LAYOUT
==============================*/
.container {
  max-width:900px;
  margin:auto;
  padding:30px;
}

/* =============================
   BUTTONS
==============================*/
.btn {
  padding:12px 20px;
  border:none;
  border-radius:8px;
  cursor:pointer;
  margin-top:10px;
  font-weight:bold;
  transition:0.2s;
  font-size:15px;
}

.btn:hover {
  transform:scale(1.05);
}

.btn:disabled {
  opacity:0.6;
  cursor:not-allowed;
  transform:none;
}

.btn.primary {
  background:#3a86ff;
  color:white;
}

.btn.accent {
  background:#2ec4b6;
  color:black;
}

.btn.warn {
  background:#ffbe0b;
  color:black;
}

/* =============================
   LOADER TEXT
==============================*/
.loader {
  margin-top:10px;
  font-size:14px;
  color:#ffe066;
  animation: pulse 1.5s infinite;
}

@keyframes pulse {
  0% {opacity: 0.3;}
  50% {opacity: 1;}
  100% {opacity: 0.3;}
}

/* =============================
   IMAGES
==============================*/
.preview-img,
.annotated-img {
  width:100%;
  margin-top:20px;
  border-radius:10px;
  box-shadow: 0px 0px 12px #0008;
}

/* =============================
   CROPPED DAMAGE GRID
==============================*/
.crop-grid {
  display: grid;
  grid-template-columns: repeat(auto-fill, minmax(140px, 1fr));
  gap: 14px;
  margin-top: 12px;
}

.crop-img {
  width: 100%;
  height: 130px;
  object-fit: cover;
  border-radius: 8px;
  border: 2px solid #3a86ff55;
  transition: 0.25s;
}

.crop-img:hover {
  transform: scale(1.06);
  border-color: #3a86ff;
}

/* =============================
   OUTPUT BOXES
==============================*/
.cost-box {
  background:#1c2541;
  padding:20px;
  border-radius:10px;
  margin-top:20px;
  white-space:pre-wrap;
  border:1px solid #3a86ff;
  box-shadow: 0px 0px 12px #0006;
  line-height: 1.5;
}

/* Part Identification Box Styling */
#partsOutput {
  background: #172036;
  border: 1px solid #ffbe0b99;
  box-shadow: 0px 0px 10px #ffbe0b44;
  margin-top: 20px;
  padding: 18px;
  border-radius: 10px;
}

/* =============================
   TITLES & TEXT
==============================*/
.page-title {
  font-size: 28px;
  margin-bottom: 20px;
  font-weight: bold;
  color: #d7e3ff;
}

h3 {
  margin-top: 25px;
  color: #a3bffa;
}

.det-list div {
  background: #1c2541;
  margin: 6px 0;
  padding: 8px 12px;
  border-radius: 6px;
  border-left: 4px solid #3a86ff;
}

/* =============================
   UPLOAD BOX
==============================*/
.upload-box {
  padding:20px;
  border:2px dashed #5bc0be;
  text-align:center;
  margin:20px 0;
  border-radius: 12px;
  background: #0f1936;
  transition: 0.25s;
}

.upload-box:hover {
  background:#12204a;
  border-color:#2ec4b6;
}


In [ ]:
%%writefile requirements.txt
flask==2.2.5
pyngrok==5.1.0
ultralytics==8.1.0
transformers==4.34.0
accelerate==0.22.0
bitsandbytes==0.39.0
pillow==10.0.0


In [ ]:
!pkill -f flask || echo "No flask running"
!pkill -f ngrok || echo "No ngrok running"

In [ ]:
!lsof -i :8000

In [ ]:
!kill -9 5206

In [ ]:
!nohup python app.py > flask.log 2>&1 &


📘  Ngrok Setup

Ngrok provides a public HTTPS link.

Your ngrok token was removed for safety.

To use ngrok:
1. Get token → https://dashboard.ngrok.com/get-started/your-authtoken  
2. Add inside notebook:

conf.get_default().auth_token = "YOUR_NGROK_TOKEN_HERE"

3. Start tunnel:

public_url = ngrok.connect(8000)

Shareable app link appears here.


In [ ]:
from pyngrok import ngrok, conf
conf.get_default().auth_token = "YOUR_NGROK_TOKEN_HERE"   # 🔑 replace with your ngrok token

public_url = ngrok.connect(8000)
print("🌍 Public URL:", public_url)

!sleep 3 && tail -n 30 flask.log


In [ ]:
!tail -n 50 flask.log